In [3]:
# # create model class
# import torch
# import torch.nn as nn

# class Model(nn.Module):

#   def __init__(self, num_features):

#     super().__init__()

#     self.linear1 = nn.Linear(num_features, 3)
#     self.relu = nn.ReLU()

#     self.linear2 = nn.Linear(3, 1)
#     self.sigmoid = nn.Sigmoid()

#   def forward(self, features):

#     out = self.linear1(features)
#     out = self.relu(out)
#     out = self.linear2(out)
#     out = self.sigmoid(out)

#     return out

In [10]:
# USING SEQUENTIAL CONTAINER
import torch
import torch.nn as nn

class Model(nn.Module):
  def __init__(self, num_features):
    super().__init__()

    self.network = nn.Sequential(
      nn.Linear(num_features, 3),
      nn.ReLU(),

      nn.Linear(3, 1),
      nn.Sigmoid()
    )

  def forward(self, features):
    out = self.network(features)


    return out

In [11]:
# create dataset
features = torch.rand(10,5)

# create model
model = Model(features.shape[1])

# call model for forward pass
# model.forward(features) # pytorch doesn't recommend this
model(features)

tensor([[0.4556],
        [0.4672],
        [0.4181],
        [0.4176],
        [0.3965],
        [0.3802],
        [0.4205],
        [0.4340],
        [0.4057],
        [0.3925]], grad_fn=<SigmoidBackward0>)

In [6]:
# SHOW MODEL WEIGHTS
model.linear1.weight

Parameter containing:
tensor([[-0.1738, -0.4175, -0.3181,  0.3972, -0.3509],
        [-0.2877,  0.1293,  0.1234, -0.3783, -0.2935],
        [ 0.3942, -0.1391,  0.2631, -0.0591,  0.0378]], requires_grad=True)

In [7]:
model.linear2.bias

Parameter containing:
tensor([0.3884], requires_grad=True)

In [ ]:
!pip install torchinfo

In [8]:
from torchinfo import summary
summary(model, input_size=(10,5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 3]                   18
├─ReLU: 1-2                              [10, 3]                   --
├─Linear: 1-3                            [10, 1]                   4
├─Sigmoid: 1-4                           [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

# Improving our training pipeline

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.shape

(569, 33)

In [4]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [5]:
df.drop(columns=["id", "Unnamed: 32"], inplace=True)

In [6]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [7]:
df.columns

Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [9]:
X_train

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
69,12.780,16.49,81.37,502.5,0.09831,0.05234,0.03653,0.02864,0.1590,0.05653,...,13.46,19.76,85.67,554.9,0.1296,0.07061,0.10390,0.05882,0.2383,0.06410
499,20.590,21.24,137.80,1320.0,0.10850,0.16440,0.21880,0.11210,0.1848,0.06222,...,23.86,30.76,163.20,1760.0,0.1464,0.35970,0.51790,0.21130,0.2480,0.08999
163,12.340,22.22,79.85,464.5,0.10120,0.10150,0.05370,0.02822,0.1551,0.06761,...,13.58,28.68,87.36,553.0,0.1452,0.23380,0.16880,0.08194,0.2268,0.09082
466,13.140,20.74,85.98,536.9,0.08675,0.10890,0.10850,0.03510,0.1562,0.06020,...,14.80,25.46,100.90,689.1,0.1351,0.35490,0.45040,0.11810,0.2563,0.08174
291,14.960,19.10,97.03,687.3,0.08992,0.09823,0.05940,0.04819,0.1879,0.05852,...,16.25,26.19,109.10,809.8,0.1313,0.30300,0.18040,0.14890,0.2962,0.08472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,12.650,18.17,82.69,485.6,0.10760,0.13340,0.08017,0.05074,0.1641,0.06854,...,14.38,22.15,95.29,633.7,0.1533,0.38420,0.35820,0.14070,0.3230,0.10330
459,9.755,28.20,61.68,290.9,0.07984,0.04626,0.01541,0.01043,0.1621,0.05952,...,10.67,36.92,68.03,349.9,0.1110,0.11090,0.07190,0.04866,0.2321,0.07211
142,11.430,17.31,73.66,398.0,0.10920,0.09486,0.02031,0.01861,0.1645,0.06562,...,12.78,26.76,82.66,503.0,0.1413,0.17920,0.07708,0.06402,0.2584,0.08096
160,11.750,20.18,76.10,419.8,0.10890,0.11410,0.06843,0.03738,0.1993,0.06453,...,13.32,26.21,88.91,543.9,0.1358,0.18920,0.19560,0.07909,0.3168,0.07987


In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
X_train

array([[-0.37790016, -0.64102802, -0.43303382, ..., -0.85734565,
        -0.81056111, -1.14173019],
       [ 1.83609403,  0.44731012,  1.88882878, ...,  1.4607899 ,
        -0.65787407,  0.32762043],
       [-0.50263222,  0.67185146, -0.49557557, ..., -0.505855  ,
        -0.99158183,  0.37472591],
       ...,
       [-0.76060082, -0.45314648, -0.75026864, ..., -0.77829066,
        -0.49416839, -0.18486455],
       [-0.66988659,  0.20443887, -0.64987266, ..., -0.54918322,
         0.42510202, -0.24672597],
       [ 0.98848294,  0.9261789 ,  1.11528604, ...,  1.28291617,
        -0.93963675,  2.1670045 ]])

In [12]:
y_train

,diagnosis
69,B
499,M
163,B
466,B
291,B
...,...
496,B
459,B
142,B
160,B


## label encoding

In [13]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [14]:
y_train

array([0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,

## Numpy arrays to PyTorch tensors

In [15]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [16]:
X_train_tensor

tensor([[-0.3779, -0.6410, -0.4330,  ..., -0.8573, -0.8106, -1.1417],
        [ 1.8361,  0.4473,  1.8888,  ...,  1.4608, -0.6579,  0.3276],
        [-0.5026,  0.6719, -0.4956,  ..., -0.5059, -0.9916,  0.3747],
        ...,
        [-0.7606, -0.4531, -0.7503,  ..., -0.7783, -0.4942, -0.1849],
        [-0.6699,  0.2044, -0.6499,  ..., -0.5492,  0.4251, -0.2467],
        [ 0.9885,  0.9262,  1.1153,  ...,  1.2829, -0.9396,  2.1670]],
       dtype=torch.float64)

In [17]:
y_train_tensor.shape

torch.Size([455])

In [18]:
X_train_tensor.shape

torch.Size([455, 30])

## Defining the model

In [28]:
class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):
    out = self.linear(features)
    out = self.sigmoid(out)

    return out




## Important Parameters

In [29]:
learning_rate = 0.1
epochs = 100
X_train_tensor = X_train_tensor.float()     # 👈 make 32-bit
y_train_tensor = y_train_tensor.float()

In [33]:
loss_function = nn.BCELoss()

In [37]:
# create Model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loop
for epoch in range(epochs):
  # forward pass
  y_pred = model(X_train_tensor)

  # loss calculate
  loss = loss_function(y_pred, y_train_tensor.view(-1, 1))

   # zero gradients
  optimizer.zero_grad()

  # backward pass
  loss.backward() # using autogard

  # parameters update
  optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, loss: {loss.item()}')

Epoch: 1, loss: 0.9158612489700317
Epoch: 2, loss: 0.647909939289093
Epoch: 3, loss: 0.5121458768844604
Epoch: 4, loss: 0.4352574050426483
Epoch: 5, loss: 0.3852017819881439
Epoch: 6, loss: 0.3495374619960785
Epoch: 7, loss: 0.32256942987442017
Epoch: 8, loss: 0.30130407214164734
Epoch: 9, loss: 0.2840051054954529
Epoch: 10, loss: 0.26959049701690674
Epoch: 11, loss: 0.25734788179397583
Epoch: 12, loss: 0.24678733944892883
Epoch: 13, loss: 0.2375599592924118
Epoch: 14, loss: 0.2294098138809204
Epoch: 15, loss: 0.22214460372924805
Epoch: 16, loss: 0.2156166285276413
Epoch: 17, loss: 0.20971046388149261
Epoch: 18, loss: 0.2043343186378479
Epoch: 19, loss: 0.19941435754299164
Epoch: 20, loss: 0.19489029049873352
Epoch: 21, loss: 0.19071240723133087
Epoch: 22, loss: 0.18683932721614838
Epoch: 23, loss: 0.18323618173599243
Epoch: 24, loss: 0.1798734962940216
Epoch: 25, loss: 0.1767260730266571
Epoch: 26, loss: 0.17377226054668427
Epoch: 27, loss: 0.17099334299564362
Epoch: 28, loss: 0.16837

In [22]:
model.linear.weight

Parameter containing:
tensor([[ 0.0123,  0.0314,  0.1040, -0.0983, -0.1003, -0.0825, -0.0178, -0.0803,
          0.0728,  0.1426,  0.1272, -0.0106, -0.0038, -0.0968, -0.0050,  0.1107,
          0.0676, -0.1007, -0.0113, -0.0864, -0.0721, -0.0384,  0.0503,  0.0917,
          0.0906, -0.0623, -0.0084,  0.1284, -0.0565, -0.0554]],
       requires_grad=True)

In [23]:
model.linear.bias

Parameter containing:
tensor([-0.4285], requires_grad=True)

## Evaluation

In [38]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor.float())
  y_pred = (y_pred > 0.5).float()

  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5497075915336609
